<a href="https://colab.research.google.com/github/Marcin19721205/BDCaseStudy/blob/main/BDCSZad11.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import pandas as pd
import numpy as np


In [5]:
#wczytanie
movies_path = "sample_data/movies.dat"
ratings_path = "sample_data/ratings.dat"

movies = pd.read_csv(movies_path, sep=r"::", engine="python", header=None, names=["movieId", "title", "genres"])
ratings = pd.read_csv(ratings_path, sep=r"::", engine="python", header=None, names=["userId", "movieId", "rating", "timestamp"])

# check
print("movies.shape:", movies.shape)
print("ratings.shape:", ratings.shape)
print("movies.head():\n", movies.head())
print("ratings.head():\n", ratings.head())


movies.shape: (19967, 3)
ratings.shape: (36504, 4)
movies.head():
    movieId                                              title  \
0        8      Edison Kinetoscopic Record of a Sneeze (1894)   
1       10                La sortie des usines Lumière (1895)   
2       12                      The Arrival of a Train (1896)   
3       25  The Oxford and Cambridge University Boat Race ...   
4       91                         Le manoir du diable (1896)   

              genres  
0  Documentary|Short  
1  Documentary|Short  
2  Documentary|Short  
3                NaN  
4       Short|Horror  
ratings.head():
    userId  movieId  rating   timestamp
0       1   114508       8  1381006850
1       2   499549       9  1376753198
2       2  1305591       8  1376742507
3       2  1428538       1  1371307089
4       3    75314       1  1595468524


In [6]:
# wczytanie movies.dat i ratings.dat z sample_data/, usun timestamp

movies = pd.read_csv("sample_data/movies.dat", sep=r"::", engine="python", header=None,
                     names=["movieId", "title", "genres"])
ratings = pd.read_csv("sample_data/ratings.dat", sep=r"::", engine="python", header=None,
                      names=["userId", "movieId", "rating", "timestamp"])

ratings = ratings.drop(columns=["timestamp"])

print("n_movies:", movies["movieId"].nunique())
print("n_ratings:", len(ratings))
print("n_users:", ratings["userId"].nunique())
print("n_items:", ratings["movieId"].nunique())


n_movies: 38013
n_ratings: 921398
n_users: 71707
n_items: 38013


In [7]:
# top-N najczęściej ocenianych filmów i filtr ratings tylko do nich
# był problem z RAM

TOP_N_ITEMS = 3000

top_items = ratings["movieId"].value_counts().head(TOP_N_ITEMS).index
ratings_f = ratings[ratings["movieId"].isin(top_items)].copy()

print("TOP_N_ITEMS:", TOP_N_ITEMS)
print("after_filter_n_ratings:", len(ratings_f))
print("after_filter_n_users:", ratings_f["userId"].nunique())
print("after_filter_n_items:", ratings_f["movieId"].nunique())


TOP_N_ITEMS: 3000
after_filter_n_ratings: 737462
after_filter_n_users: 65452
after_filter_n_items: 3000


In [8]:
# fmacierz user–item (pivot) z brakami jako 0 oraz opcjonalny mean-centering per user. #
user_item = ratings_f.pivot_table(index="userId", columns="movieId", values="rating", aggfunc="mean").fillna(0)

MEAN_CENTER = True
if MEAN_CENTER:
    user_means = user_item.replace(0, np.nan).mean(axis=1)
    user_item_mc = user_item.sub(user_means, axis=0).where(user_item != 0, 0)
else:
    user_item_mc = user_item

print("user_item.shape:", user_item.shape)
print("nonzero:", int((user_item != 0).sum().sum()))
print("density:", float((user_item != 0).sum().sum() / (user_item.shape[0] * user_item.shape[1])))
print("MEAN_CENTER:", MEAN_CENTER)


user_item.shape: (65452, 3000)
nonzero: 737229
density: 0.0037545529548371325
MEAN_CENTER: True


In [9]:
user_item.head()

movieId,12349,15864,17136,21749,22100,25316,27977,31381,32138,32553,...,11388580,11390036,11464826,11561866,11833648,12361974,12676326,12801262,13024674,15097216
userId,,,,,,,,,,,,,,,,,,,,,
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [10]:
# macierz podobieństw item×item metodą cosine na wektorach ocen #
from scipy.sparse import csr_matrix
from sklearn.metrics.pairwise import cosine_similarity

X = csr_matrix(user_item_mc.values)  # (n_users, n_items)
item_sim = cosine_similarity(X.T, dense_output=True)  # (n_items, n_items)

np.fill_diagonal(item_sim, 1.0)

item_ids = user_item_mc.columns.to_numpy()

print("item_sim.shape:", item_sim.shape)
print("item_sim_minmax:", float(item_sim.min()), float(item_sim.max()))
print("item_ids_sample:", item_ids[:10])


item_sim.shape: (3000, 3000)
item_sim_minmax: -0.8258066549090047 1.0
item_ids_sample: [12349 15864 17136 21749 22100 25316 27977 31381 32138 32553]


In [11]:
# score dla nieocenionych filmów jako suma(sim * rating) po filmach ocenionych przez usera
user_counts = ratings_f["userId"].value_counts()
rank = 1
user_id = int(user_counts.index[rank - 1])

u_row = user_item_mc.loc[user_id].values
rated_mask = user_item.loc[user_id].values != 0

K = 30
scores = np.zeros_like(u_row, dtype=float)

rated_idx = np.where(rated_mask)[0]
rated_vals = u_row[rated_idx]

for i in range(len(scores)):
    if rated_mask[i]:
        scores[i] = -np.inf
        continue
    sims = item_sim[i, rated_idx]
    if K is not None and len(sims) > K:
        topk = np.argpartition(sims, -K)[-K:]
        scores[i] = float(np.dot(sims[topk], rated_vals[topk]))
    else:
        scores[i] = float(np.dot(sims, rated_vals))

rec = pd.DataFrame({"movieId": item_ids, "score": scores})
rec = rec.sort_values("score", ascending=False).reset_index(drop=True)

print("user_id:", user_id, "| n_rated:", int(rated_mask.sum()), "| K:", K)
print(rec.head(10))


user_id: 17405 | n_rated: 1268 | K: 30
   movieId     score
0  2267998  7.011164
1  2024544  6.509151
2   790636  6.068250
3  1454468  5.799325
4  1631867  5.100387
5  4154756  5.026296
6   111161  5.004355
7  3498820  4.982834
8   468569  4.835521
9  2380307  4.803243


In [23]:
#visualizar_recomendacion(user_id, N=5), item-based score i daje TOP-N z tytułami. #

movie_titles = movies.drop_duplicates("movieId").set_index("movieId")["title"]

def visualizar_recomendacion(user_id: int, N: int = 5, K: int = 30):
    if user_id not in user_item.index:
        print(f"user_id={user_id} -> brak w macierzy user_item (po filtrze)")
        return

    u_row_mc = user_item_mc.loc[user_id].values
    rated_mask = user_item.loc[user_id].values != 0

    rated_idx = np.where(rated_mask)[0]
    rated_vals = u_row_mc[rated_idx]

    scores = np.full(len(item_ids), -np.inf, dtype=float)

    for i in range(len(item_ids)):
        if rated_mask[i]:
            continue
        sims = item_sim[i, rated_idx]
        if K is not None and len(sims) > K:
            topk = np.argpartition(sims, -K)[-K:]
            scores[i] = float(np.dot(sims[topk], rated_vals[topk]))
        else:
            scores[i] = float(np.dot(sims, rated_vals))

    out = pd.DataFrame({"movieId": item_ids, "score": scores})
    out = out[np.isfinite(out["score"])]
    out = out.sort_values("score", ascending=False).head(N).copy()

    out["title"] = out["movieId"].map(movie_titles)
    out = out[["movieId", "title", "score"]].reset_index(drop=True)

    print(f"user_id={user_id} | TOP-{N} | K={K}")
    print("------------------------")
    print(out.to_string(index=False))
    print("------------------------")

print(visualizar_recomendacion(user_id, N=5, K=30))


user_id=17405 | TOP-5 | K=30
------------------------
 movieId                     title    score
 2267998          Gone Girl (2014) 7.011164
 2024544   12 Years a Slave (2013) 6.509151
  790636 Dallas Buyers Club (2013) 6.068250
 1454468            Gravity (2013) 5.799325
 1631867   Edge of Tomorrow (2014) 5.100387
------------------------
None


In [24]:
#shape i gęstość macierzy, ile ocen ma user, oraz top-10 podobnych filmów do wybranego filmu. #

import numpy as np
import pandas as pd

nz = int((user_item != 0).sum().sum())
all_ = int(user_item.shape[0] * user_item.shape[1])
density = nz / all_ if all_ else 0.0

print("user_item.shape:", user_item.shape)
print("nonzero:", nz, "| all:", all_, "| density:", float(density))

n_user_ratings = int((user_item.loc[user_id] != 0).sum()) if user_id in user_item.index else 0
print("user_id:", user_id, "| n_user_ratings_in_matrix:", n_user_ratings, "| ok>=5:", n_user_ratings >= 5)

i0 = int(np.argmax((user_item != 0).sum(axis=0).values))
movie0_id = int(item_ids[i0])

sim_row = item_sim[i0].copy()
sim_row[i0] = -np.inf
top10_idx = np.argsort(sim_row)[-10:][::-1]

top10 = pd.DataFrame({
    "movieId": item_ids[top10_idx],
    "title": pd.Series(item_ids[top10_idx]).map(movie_titles).values,
    "sim": sim_row[top10_idx],
}).reset_index(drop=True)

print("sanity_movieId:", movie0_id, "| title:", movie_titles.get(movie0_id, None))
print("top10_similar_movies:\n", top10.to_string(index=False))


user_item.shape: (65452, 3000)
nonzero: 737229 | all: 196356000 | density: 0.0037545529548371325
user_id: 17405 | n_user_ratings_in_matrix: 1268 | ok>=5: True
sanity_movieId: 1454468 | title: Gravity (2013)
top10_similar_movies:
  movieId                           title      sim
 1535109         Captain Phillips (2013) 0.189990
 1979320                     Rush (2013) 0.188613
 2024544         12 Years a Slave (2013) 0.178952
  993846  The Wolf of Wall Street (2013) 0.156216
 2582802                 Whiplash (2014) 0.139882
 2278388 The Grand Budapest Hotel (2014) 0.132795
 1392214                Prisoners (2013) 0.132556
 1631867         Edge of Tomorrow (2014) 0.130602
  790636       Dallas Buyers Club (2013) 0.127042
 2267998                Gone Girl (2014) 0.125834
